In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import itertools
import torch

from tsvar.simulate import GrangerBuscaSimulator
from tsvar.wold_model import VariationalWoldModel

---

## Generate small toy example dataset

Define model parameters

In [4]:
dim = 3

baseline = np.array([0.1, 0.1, 0.2])

# adjacency[i,j] = magnitude of influence from i to j
adjacency = np.array([
    [0.3, 0.8, 0.1],
    [0.2, 0.3, 0.1],
    [0.2, 0.1, 0.0]
])

end_time = 100.0

for i, j in itertools.product(range(dim), repeat=2):
    print(f"w({j:d} -> {i:d}) = {adjacency[j,i]:.2f}")

w(0 -> 0) = 0.30
w(1 -> 0) = 0.20
w(2 -> 0) = 0.20
w(0 -> 1) = 0.80
w(1 -> 1) = 0.30
w(2 -> 1) = 0.10
w(0 -> 2) = 0.10
w(1 -> 2) = 0.10
w(2 -> 2) = 0.00


Simulate a dataset

In [5]:
wold_sim = GrangerBuscaSimulator(mu_rates=baseline, Alpha_ba=adjacency, Beta_b=np.ones((dim,)))
events = wold_sim.simulate(end_time, seed=42)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]


print(list(map(len, events)))
print()
print(events[0])
print()
print(events[1])
print()
print(events[2])

[35, 47, 18]

tensor([ 2.5502,  3.3542,  9.4456, 10.5112, 10.9384, 12.8847, 14.4720, 17.2495,
        26.6435, 26.9533, 27.0346, 27.1342, 28.0862, 28.5891, 36.3908, 36.7676,
        38.7048, 42.1140, 42.8806, 44.1329, 49.5418, 62.3604, 63.0562, 66.4042,
        69.0349, 70.9969, 73.9548, 75.0063, 79.1659, 79.6696, 80.8179, 81.0922,
        84.0139, 88.4186, 92.6550])

tensor([10.9159, 11.6606, 11.8246, 14.0996, 17.1532, 19.1170, 20.6203, 21.8233,
        23.4265, 25.3780, 29.9809, 30.1367, 30.2652, 33.5775, 34.3999, 39.7479,
        41.8163, 49.4462, 49.6672, 50.5802, 50.6303, 54.5108, 58.2400, 59.4649,
        59.7941, 59.9094, 65.5344, 67.7530, 69.0344, 72.8742, 72.9334, 74.4263,
        75.4724, 75.6039, 78.6012, 79.0697, 79.5100, 79.7112, 80.9577, 82.4931,
        84.0964, 84.4289, 85.2517, 88.8507, 90.3975, 90.6006, 91.0759])

tensor([ 5.7092, 21.3760, 28.9135, 36.1351, 36.9744, 43.6678, 58.2271, 60.4763,
        65.2137, 69.0521, 70.6834, 72.7963, 74.6422, 79.8425, 82.9051, 91.34

---

## Test `_init_variational_cache()`

Create object, set the dataset, and init the variational cache.

In [6]:


alpha_shp = 1.0 * np.ones((dim + 1, dim))
alpha_rate = 0.5 * np.ones((dim + 1, dim))
alpha_rate[0,:] = 10.0
z_param = [1.0 * np.ones((len(events[i]), dim)) for i in range(dim)]

self = VariationalWoldModel(verbose=True)
self.set_data(events)

self._as_pr = alpha_shp
self._ar_pr = alpha_rate
self._zc_pr = z_param
vals_numba = _compute_C_ik(asp=self._as_pr, arp=self._ar_pr,
                   delta_ikj=self.delta_ikj,
                   events=self.events)

AttributeError: 'VariationalWoldModel' object has no attribute '_compute_C_ik'

Do the same in pure python implementation with optimisation through `scipy`

In [ ]:
from scipy.optimize import newton


def solve_halley(func, fprime, fprime2, x0, max_iter, tol):
    x = float(x0)
    for it in range(max_iter):
        f = func(x)
        fp = fprime(x)
        fpp = fprime2(x)
        print(f, x)
        x_new = x - (2 * f * fp) / (2 * fp**2 - f * fpp)
        if abs(x - x_new) < tol:
            return x
        x = x_new
    raise RuntimeError('Did not converged')

    
def worker(i, k):
    a = self._as_pr[:,i]
    delta_ik = self.delta_ikj[i][k,:]
    b = (1 / (1 + delta_ik)) / self._ar_pr[:,i]
    ab3 = a * b**3
    uki = np.sum(a * b)

    def func(x):
        return float(np.sum(ab3 / (b + x)**2) - uki / 4)

    def fprime(x):
        return float(-2 * np.sum(ab3 / (b + x)**3))

    def fprime2(x):
        return float(6 * np.sum(ab3 / (b + x)**4))

    x0 = float(b.max())

    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html
    xhat, info = newton(func=func, x0=x0, fprime=fprime, 
                        fprime2=fprime2, maxiter=10, tol=1e-3,
                        full_output=True)
    
    if not info.converged:
        raise RuntimeException(f'item (i={i:d}, k={k:d}) did not converged')
    
    return xhat


def compute_all_vals():
    vals = [torch.ones_like(events[i]) for i in range(dim)]
    # dim i
    for i in range(self.dim):
        # event k in dim i
        for k in range(len(events[i])): 
            if (k+1) % 1000 == 0:
                print(f"\rProcess dim {i:d}, event {k+1:>6d}", end="")   
            vals[i][k] = worker(i,k)
    return vals
            
vals_python = compute_all_vals()

Test that both implementations give the same result.

In [ ]:
[np.allclose(a,b,atol=1e-3) for a, b in zip(vals_numba, vals_python)]

Plot one of the optimized values computed in the cache.

In [ ]:
i, k = 1, 11

a = self._as_pr[:,i]
delta_ik = self.delta_ikj[i][k,:]
b = (1 / (1 + delta_ik)) / self._ar_pr[:,i]
ab3 = a * b**3
uki = np.sum(a * b)
def func(x):
    return float(np.sum(ab3 / (b + x)**2) - uki / 4)
xhat = vals_python[i][k]

xx = np.linspace(1e-5, 10.0, 100)
yy = [func(x) for x in xx]

%matplotlib inline
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 6))
plt.axhline(0, ls=':', c='k', label='Zero line')
if xhat:
    plt.axvline(xhat, ls='--', c='C3', label='x opt')
plt.plot(xx, yy, label='func')
plt.legend();